In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import datetime
import random 
pd.set_option('display.max_columns', 5000)

In [2]:
prepend = '' + 'NE_'
combined_data = pd.read_csv('../data/processed/' + prepend + 'multi_var_unscaled.csv',  index_col=0)
print(combined_data.shape)
combined_data.head()

(136, 32)


,1 Month Prior Intra-Mobility,1 Month Prior Inter-Mobility,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Post Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility,2wk Post Inter-Mobility,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Total_age65plus,Total households!!Average household size,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,POP_ESTIMATE_2018,HospCt,NursingCt,Hispanic Population,Black Population,Outbreak Month Temp AVG / F,Time from outbreak to intervention,Time from outbreak to national intervention,Time from national intervention to outbreak,Deaths,normalized_deaths
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9001.0,0.074978,0.104549,0.053966,0.062702,0.053966,0.069927,0.084243,0.069927,1467.2,2.0,10.1,47.4,4.0,119.4,0.158841,2.68,0.017835,0.011929,0.104794,0.060231,0.068122,943823.0,0.000006,0.000093,0.201771,0.127248,36.9,-4.0,21,0,178.0,0.000189
9003.0,0.080955,0.110928,0.064932,0.063348,0.064932,0.081730,0.081377,0.081730,1216.2,1.0,10.5,37.8,4.2,95.6,0.171250,2.50,0.014742,0.011147,0.094277,0.055061,0.065381,892697.0,0.000007,0.000131,0.184533,0.156606,35.3,-15.0,10,0,369.0,0.000413
9005.0,0.068432,0.106744,0.060970,0.060930,0.060970,0.085063,0.086268,0.085063,206.3,4.0,7.4,35.1,3.8,99.9,0.212588,2.42,0.009138,0.011258,0.084219,0.051063,0.050273,181111.0,0.000017,0.000160,0.067252,0.022771,47.2,-19.0,6,0,69.0,0.000381
9007.0,0.072337,0.118761,0.062333,0.061928,0.062333,0.090481,0.091166,0.090481,448.6,1.0,5.8,41.5,3.5,116.2,0.203262,2.34,0.016505,0.009515,0.083377,0.044141,0.074993,162682.0,0.000006,0.000209,0.064310,0.054948,49.7,-18.0,7,0,57.0,0.000350
9009.0,0.084441,0.118351,0.064114,0.062205,0.064114,0.081794,0.081903,0.081794,1426.7,2.0,9.9,34.9,4.4,88.7,0.173604,2.47,0.013458,0.010612,0.090836,0.054647,0.078346,857620.0,0.000008,0.000134,0.186334,0.149594,36.2,-12.0,13,0,229.0,0.000267


In [3]:
adjacency = pd.read_excel("../data/raw/adjacent_counties.xlsx")
adjacency.head()

,County_1_Name,County_1_FIPS,County_2_Name,County_2_FIPS
0,"Autauga County, AL",1001,"Autauga County, AL",1001
1,"Autauga County, AL",1001,"Chilton County, AL",1021
2,"Autauga County, AL",1001,"Dallas County, AL",1047
3,"Autauga County, AL",1001,"Elmore County, AL",1051
4,"Autauga County, AL",1001,"Lowndes County, AL",1085


In [4]:
colnames = ['FIPS1', 'FIPS2']
for col in combined_data.columns: 
    colnames.append(col)
print(colnames)

['FIPS1', 'FIPS2', '1 Month Prior Intra-Mobility', '1 Month Prior Inter-Mobility', '2wk Prior Intra-Mobility', '2wk Onset Intra-Mobility', '2wk Post Intra-Mobility', '2wk Prior Inter-Mobility', '2wk Onset Inter-Mobility', '2wk Post Inter-Mobility', 'Density per square mile of land area - Population', 'Rural-urban_Continuum Code_2013', 'Percent of adults with less than a high school diploma 2014-18', "Percent of adults with a bachelor's degree or higher 2014-18", 'Unemployment_rate_2018', 'Med_HH_Income_Percent_of_State_Total_2018', 'Total_age65plus', 'Total households!!Average household size', 'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool', 'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten', 'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8)', 'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12)', 'SCHOOL ENROLLMENT!!Popu

In [5]:
pairwise_data = pd.DataFrame(columns=colnames)
pairwise_data.head()

,FIPS1,FIPS2,1 Month Prior Intra-Mobility,1 Month Prior Inter-Mobility,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Post Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility,2wk Post Inter-Mobility,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Total_age65plus,Total households!!Average household size,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,POP_ESTIMATE_2018,HospCt,NursingCt,Hispanic Population,Black Population,Outbreak Month Temp AVG / F,Time from outbreak to intervention,Time from outbreak to national intervention,Time from national intervention to outbreak,Deaths,normalized_deaths


In [6]:
count = 0 
for index,row in adjacency.iterrows():
    fips1 = row['County_1_FIPS']
    fips2 = row['County_2_FIPS']
    if fips1 >= fips2:  # To avoid duplicates 
        continue
    if fips1 not in combined_data.index or fips2 not in combined_data.index:
        continue
    add_row = {}
    if random.randint(0,1) == 0:    
        add_row['FIPS1'] = fips1
        add_row['FIPS2'] = fips2
    else: 
        add_row['FIPS1'] = fips2
        add_row['FIPS2'] = fips1
    # Take differences 
    found_na = False
    for col in combined_data.columns:
        add_row[col] = combined_data.loc[fips1, col] - combined_data.loc[fips2,col]
        if (np.isnan(combined_data.loc[fips1, col]) or np.isnan(combined_data.loc[fips2,col])) and not found_na:
            print(add_row[col])
            count +=1
            found_na = True 
    pairwise_data = pairwise_data.append(add_row, ignore_index=True)
#     if index % 20 == 0:
#         print(pairwise_data.head())
print(count)
pairwise_data.head()

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
145


,FIPS1,FIPS2,1 Month Prior Intra-Mobility,1 Month Prior Inter-Mobility,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Post Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility,2wk Post Inter-Mobility,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Total_age65plus,Total households!!Average household size,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,POP_ESTIMATE_2018,HospCt,NursingCt,Hispanic Population,Black Population,Outbreak Month Temp AVG / F,Time from outbreak to intervention,Time from outbreak to national intervention,Time from national intervention to outbreak,Deaths,normalized_deaths
0,9005.0,9001.0,0.006546,-0.002195,-0.007004,0.001772,-0.007004,-0.015136,-0.002025,-0.015136,1260.9,-2.0,2.7,12.3,0.2,19.5,-0.053747,0.26,0.008697,0.000671,0.020575,0.009168,0.017849,762712.0,-0.000010,-0.000067,0.134519,0.104478,-10.3,15.0,15.0,0.0,109.0,-0.000192
1,9001.0,9009.0,-0.009464,-0.013802,-0.010148,0.000497,-0.010148,-0.011867,0.002340,-0.011867,40.5,0.0,0.2,12.5,-0.4,30.7,-0.014762,0.21,0.004377,0.001317,0.013958,0.005584,-0.010224,86203.0,-0.000002,-0.000041,0.015437,-0.022346,0.7,8.0,8.0,0.0,-51.0,-0.000078
2,9001.0,36027.0,0.001519,-0.001226,-0.006075,0.004145,-0.006075,-0.007305,0.006162,-0.007305,1093.3,1.0,0.6,12.2,0.3,1.7,-0.016950,0.12,0.005381,0.002284,0.024799,0.005157,-0.014583,650105.0,-0.000004,0.000015,0.076923,0.008478,1.4,11.0,12.0,0.0,150.0,0.000093
3,9001.0,36059.0,-0.028613,-0.047064,-0.017232,-0.025353,-0.017232,-0.023406,-0.036621,-0.023406,-3237.6,1.0,1.3,2.2,0.5,-51.0,-0.018789,-0.31,0.001108,0.000860,0.009216,0.007752,-0.003610,-414520.0,-0.000002,0.000046,0.029457,-0.002621,-2.8,-1.0,0.0,0.0,-455.0,-0.000277
4,9001.0,36079.0,0.009198,-0.012585,-0.003712,0.005239,-0.003712,-0.016255,-0.002832,-0.016255,1034.3,1.0,3.1,7.5,0.3,-29.7,-0.014772,-0.05,0.004204,0.003910,0.013452,0.004139,-0.004108,844931.0,-0.000004,0.000053,0.043568,0.089884,-13.6,14.0,15.0,0.0,171.0,0.000118


In [7]:
pairwise_data.shape

(283, 34)

In [8]:
pre_partial_corr = pairwise_data.filter(["Rural-urban_Continuum Code_2013", "2wk Prior Intra-Mobility", "Total_age65plus", "HospCt", "NursingCt", "normalized_deaths"])
pre_partial_corr = pre_partial_corr.dropna()
pre_partial_corr.to_csv('../data/processed/' + prepend + 'local_pairwise_multi_var_unscaled.csv', index=False)
print(pre_partial_corr.shape)
pre_partial_corr.head()

(153, 6)


,Rural-urban_Continuum Code_2013,2wk Prior Intra-Mobility,Total_age65plus,HospCt,NursingCt,normalized_deaths
0,-2.0,-0.007004,-0.053747,-0.000010,-0.000067,-0.000192
1,0.0,-0.010148,-0.014762,-0.000002,-0.000041,-0.000078
2,1.0,-0.006075,-0.016950,-0.000004,0.000015,0.000093
3,1.0,-0.017232,-0.018789,-0.000002,0.000046,-0.000277
4,1.0,-0.003712,-0.014772,-0.000004,0.000053,0.000118


In [9]:
pairwise_data = pairwise_data.dropna()
pairwise_data.shape

(138, 34)

## Standard Deviation Scaling

In [10]:
scaler = StandardScaler() 
print(pairwise_data.to_numpy().shape)
# scaled_data = scaler.fit_transform(combined_data)
for i in range(2, len(pairwise_data.columns.values)- 2 ): # 1 to -2 so that it scales all except FIPs, deaths, and normalized deaths 
    col = pairwise_data.columns.values[i]
#     combined_data[col] = scaled_data[:, i]
    pairwise_data[col] = scaler.fit_transform(np.array(pairwise_data[col]).reshape(-1,1)).reshape(-1)

(138, 34)


In [11]:
print(pairwise_data.max(axis=0))
print(pairwise_data.min(axis=0))
pairwise_data.head()

FIPS1                                                                                                36119.000000
FIPS2                                                                                                36119.000000
1 Month Prior Intra-Mobility                                                                             3.170726
1 Month Prior Inter-Mobility                                                                             2.257652
2wk Prior Intra-Mobility                                                                                 2.600772
2wk Onset Intra-Mobility                                                                                 2.649325
2wk Post Intra-Mobility                                                                                  2.600772
2wk Prior Inter-Mobility                                                                                 2.707963
2wk Onset Inter-Mobility                                                                

,FIPS1,FIPS2,1 Month Prior Intra-Mobility,1 Month Prior Inter-Mobility,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Post Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility,2wk Post Inter-Mobility,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Total_age65plus,Total households!!Average household size,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,POP_ESTIMATE_2018,HospCt,NursingCt,Hispanic Population,Black Population,Outbreak Month Temp AVG / F,Time from outbreak to intervention,Time from outbreak to national intervention,Time from national intervention to outbreak,Deaths,normalized_deaths
0,9005.0,9001.0,0.292820,-0.136824,-0.657121,0.055272,-0.657121,-1.088320,-0.118074,-1.088320,0.331598,-2.287488,0.700833,0.978940,0.199673,0.770124,-1.834313,1.396744,1.990262,0.236255,1.463116,1.195226,0.598584,1.846255,-1.685414,-1.394913,1.404913,0.983311,-1.244017,1.881413,2.423807,0.132101,109.0,-0.000192
1,9001.0,9009.0,-0.651002,-0.697082,-0.919352,-0.034569,-0.919352,-0.854414,0.128653,-0.854414,0.172158,-0.079982,0.188571,0.996821,-0.565739,1.188067,-0.630706,1.163784,0.985465,0.434791,1.004966,0.742264,-0.165859,0.258644,-0.157633,-0.864538,0.275889,-0.069168,0.000164,0.990509,1.310446,0.132101,-51.0,-0.000078
2,9001.0,36027.0,-0.003539,-0.090060,-0.579612,0.222411,-0.579612,-0.527998,0.344690,-0.527998,0.309702,1.023771,0.270533,0.970000,0.327241,0.105892,-0.698240,0.744457,1.218992,0.731696,1.755568,0.688342,-0.284565,1.581992,-0.530701,0.272091,0.858844,0.186632,0.079339,1.372325,1.946652,0.132101,150.0,0.000093
3,9001.0,36059.0,-1.779953,-2.302684,-1.510185,-1.855509,-1.510185,-1.680108,-2.073753,-1.680108,-0.256112,1.023771,0.413966,0.075991,0.582378,-1.860682,-0.755027,-1.258994,0.225186,0.294284,0.676651,1.016260,0.014235,-0.916438,-0.145992,0.907589,0.408819,0.094525,-0.395712,-0.154940,0.038034,0.132101,-455.0,-0.000277
4,9001.0,36079.0,0.449155,-0.638365,-0.382503,0.299476,-0.382503,-1.168391,-0.163725,-1.168391,0.301994,1.023771,0.782794,0.549816,0.327241,-1.065843,-0.631013,-0.047605,0.945261,1.231275,0.969946,0.559648,0.000673,2.039204,-0.512184,1.043450,0.542603,0.862204,-1.617271,1.754141,2.423807,0.132101,171.0,0.000118


In [12]:
pairwise_data.to_csv('../data/processed/' + prepend + 'local_pairwise_multi_var_scaled.csv', index=False)